In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import gaussian_kde


# https://velog.io/@sirasony12/%EC%A7%80%EB%8F%84-%EC%8B%9C%EA%B0%81%ED%99%94

!pip install folium
import folium
from folium import plugins

import json

In [4]:
plt.rcParams['font.family'] = 'Malgun Gothic'

In [6]:
data = pd.read_csv("./data/Apart Deal Updated.csv")
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
sidoDong = pd.read_csv("./data/sidoDong_pop_latlng.csv")
sidoDong.drop(['Unnamed: 0'], axis = 1, inplace = True)

sidoDong['평당거래금액'] = sidoDong['평균거래금액']/sidoDong['평균전용면적']

FileNotFoundError: [Errno 2] No such file or directory: './data/Apart Deal Updated.csv'

In [ ]:
data

In [ ]:
sidoDong

In [ ]:
## 위도 경도의 평균값 ==> 지도의 ㄱ중심
# x_center = sidoDong['latitude'].mean()
# y_center = sidoDong['longitude'].mean()

x_center = 36.2879809
y_center = 128.0745511

In [ ]:
## 지도에 시도 구분선

with open('./data/skorea-provinces-geo.json' , mode='rt', encoding='utf-8') as f:
   geo = json.loads(f.read())
   f.close()

In [ ]:
## 평균거래금액 시각화

m = folium.Map([x_center, y_center], zoom_start = 6.8)

folium.GeoJson(geo, name='skorea-provinces-geo',
              style_function=lambda feature: {
        "fillColor": "#ffffff",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
        },
              ).add_to(m)


m.add_child(plugins.HeatMap(zip(sidoDong['latitude'],
                               sidoDong['longitude'],
                               sidoDong['평균거래금액']), radius=16))
    
m.save('평균거래금액.html')
m

In [ ]:
m2 = folium.Map([x_center, y_center], zoom_start = 6.8)

folium.GeoJson(geo, name='skorea-provinces-geo',
              style_function=lambda feature: {
        "fillColor": "#ffffff",
        "color": "black",
        "weight": 1.5,
        "dashArray": "5, 5",
        },
              ).add_to(m2)

m2.add_child(plugins.HeatMap(zip(sidoDong['latitude'],
                               sidoDong['longitude'],
                               sidoDong['거래수']), radius=16))
  
m2.save("거래수.html")
m2

In [ ]:
m3 = folium.Map([x_center, y_center], zoom_start = 6.8)

folium.GeoJson(geo, name='skorea-provinces-geo',
              style_function=lambda feature: {
        "fillColor": "#ffffff",
        "color": "black",
        "weight": 1.5,
        "dashArray": "5, 5",
        },
              ).add_to(m3)

m3.add_child(plugins.HeatMap(zip(sidoDong['latitude'],
                               sidoDong['longitude'],
                               sidoDong['인구수']), radius=16))
    
m3.save("인구수.html")
m3

In [ ]:
m4 = folium.Map([x_center, y_center], zoom_start = 6.8)


folium.GeoJson(geo, name='skorea-provinces-geo',
              style_function=lambda feature: {
        "fillColor": "#ffffff",
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        },
              ).add_to(m4)


m4.add_child(plugins.HeatMap(zip(sidoDong['latitude'],
                               sidoDong['longitude'],
                               sidoDong['평균전용면적']), radius=16))
    
m4.save("평균전용면적.html")
m4

In [ ]:
m4 = folium.Map([x_center, y_center], zoom_start = 6.8)

folium.GeoJson(geo, name='skorea-provinces-geo',
              style_function=lambda feature: {
        "fillColor": "#ffffff",
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        },
              ).add_to(m4)



m4.add_child(plugins.HeatMap(zip(sidoDong['latitude'],
                               sidoDong['longitude'],
                               sidoDong['평당거래금액']), radius=16))
    
m4.save("평당 거래금액.html")
m4

In [ ]:
sidoDong.sort_values('평당거래금액',ascending=False)

In [ ]:
### 시도별로 데이터 정리

temp = sidoDong.copy()
temp['총거래금액'] = temp['평균거래금액'] * temp['거래수']


sidoData = temp.groupby('시도', as_index=False).agg(총거래금액 = ('총거래금액','sum'), 
                                                      총거래수=('거래수','sum'),
                                                    인구수 = ('인구수','sum')
                                                  
                                                     )
sidoData['평균거래금액(만원)'] = (sidoData['총거래금액'] / sidoData['총거래수']).round(2)
sidoData['1인당평균거래금액'] = sidoData['총거래금액'] / sidoData['인구수']



# https://eis.work.go.kr/eisps/main/index.do
## 일자리 수
list_sido = ['서울','대구','광주','울산','경기','충남','전남','경남',
             '강원','부산','인천','대전','세종','충북','전북','경북','제주']
list_emp = [34102, 8288, 4043, 6018, 65503, 11499, 11546, 19885, 7319, 
            13559, 13542, 3266, 1702, 10051, 11425, 13921, 2957]
dic = {'시도' : list_sido, '일자리수' : list_emp}
empData = pd.DataFrame(dic)
sidoData = sidoData.merge(empData, how="inner")

## 인구대비 일자리수
sidoData['인구대비일자리'] = (sidoData['일자리수'] / sidoData['인구수']) * 100


# https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1DA7035
##  시도별 평균 임금
sidoYeonbong = pd.read_csv("./data/행정구역_시도___산업_규모별_임금_및_근로시간_상용근로자_상용근로자_5인이상_사업체__20240102121032.csv", encoding='euc-kr')
sidoYeonbong.drop(columns = ['산업분류별', '규모별', '2019', '2019.1', '2019.2', '2019.3', '2019.5', '2019.6', '2019.7'], inplace=True)
sidoYeonbong.columns=['시도','상용월급여액(만원)']

sidoYeonbong.drop(0,axis=0, inplace=True)
sidoYeonbong.iloc[0]['시도'] = '전국'

sidoYeonbong['상용월급여액(만원)'] = sidoYeonbong['상용월급여액(만원)'].astype(int)
sidoYeonbong['상용월급여액(만원)'] = (sidoYeonbong['상용월급여액(만원)']/10000).round(2)
sidoYeonbong['연봉(만원)'] = sidoYeonbong['상용월급여액(만원)']*12

totalYeonbong = sidoYeonbong.iloc[0]['연봉(만원)']
sidoYeonbong.drop(1,axis=0, inplace=True)

sidoData = sidoData.merge(sidoYeonbong, how="inner")


### 시도별 평균거래금액/ 평균연봉
sidoData['연봉대비거래금액'] = (sidoData['평균거래금액(만원)'])/sidoData['연봉(만원)']


sidoCopy = sidoData.copy()
sidoData.to_csv("./data/sidoManyData.csv")
sidoData



In [ ]:
plt.figure(figsize=(10,4))

sidoCopy.sort_values('평균거래금액(만원)', ascending=False, inplace=True)

plt.bar(sidoCopy['시도'], sidoCopy['평균거래금액(만원)']/10000)

plt.xticks(rotation=70) 
plt.title("시도별 평균거래금액")
plt.ylabel("평균거래금액(억)")
plt.grid(True, color = 'grey', linestyle=":")

plt.show()

In [ ]:
plt.figure(figsize=(10,4))


sidoCopy.sort_values('총거래수', ascending=False, inplace=True)

plt.bar(sidoCopy['시도'], sidoCopy['총거래수']/10000)
plt.xticks(rotation=70) 
plt.title("시도별 거래수 (만 건)")
plt.ylabel("거래수 (만 건)")
plt.yticks(range(0,150,30))
plt.grid(True, color = 'grey', linestyle=":")
plt.show()

In [ ]:
plt.figure(figsize=(10,4))


sidoCopy.sort_values('인구수', ascending=False, inplace=True)

plt.bar(sidoCopy['시도'], sidoCopy['인구수']/10000)
plt.xticks(rotation=70) 
plt.title("시도별 인구수")
plt.ylabel("인구수 (만 명)")
plt.grid(True, color = 'grey', linestyle=":")
plt.show()



In [ ]:
# temp = sidoDong.copy()
# temp['총거래금액'] = temp['평균거래금액'] * temp['거래수']

# sidoPricePer = temp.groupby('시도', as_index=False).agg(시도별총거래금액 = ('총거래금액','sum'), 총인구 = ('인구수','sum'))

# sidoPricePer['1인당평균거래금액'] = sidoPricePer['시도별총거래금액'] / sidoPricePer['총인구']

plt.figure(figsize=(10,4))


sidoCopy.sort_values('1인당평균거래금액', ascending=False, inplace=True)


plt.bar(sidoCopy['시도'], sidoCopy['1인당평균거래금액'])
plt.xticks(rotation=70) 
plt.title("시도별 1인당 평균거래금액")
plt.ylabel("평균거래금액(만 원)")
plt.grid(True, color = 'grey', linestyle=":")


plt.show()




## 시도 별 일자리 수 

In [ ]:


sidoCopy.sort_values('1인당평균거래금액', ascending=False, inplace=True)



list_explode = [0 for i in range(sidoCopy['일자리수'].size)]
list_explode[0] = 0.1
list_explode[1] = 0.1
list_explode[-1] = 0.1

colors = sns.color_palette("Set2")
plt.pie(sidoCopy['일자리수'],labels=sidoCopy['시도'] ,
        autopct="%.1f%%", colors = colors, explode = list_explode , radius=1.5)
plt.title('시도별 일자리 수', pad = 60)
plt.legend(bbox_to_anchor=(1.5, 1.0), loc="upper left")

plt.savefig("./data/시도별일자리수.jpeg")
plt.show()


In [ ]:

sidoCopy.sort_values('1인당평균거래금액', ascending=False, inplace=True)

colors = sns.color_palette("Set2")
plt.pie(sidoCopy['인구대비일자리'],labels=sidoCopy['시도'] ,
        autopct="%.1f%%", colors = colors, explode = list_explode , radius=1.5)
plt.title('시도별 인구대비 일자리 수', pad = 60)
plt.legend(bbox_to_anchor=(1.5, 1.0), loc="upper left")


plt.show()

In [ ]:
### 시도별 평균연봉

plt.figure(figsize=(10,4))

sidoCopy.sort_values('연봉(만원)', ascending=False, inplace=True)

plt.bar(sidoCopy['시도'], sidoCopy['연봉(만원)'])

plt.xticks(rotation=70) 
plt.title("시도 별 평균 연봉")
plt.ylabel("평균 연봉 (만원)")
plt.ylim(2000)
plt.yticks(range(2000,5000,500))
plt.hlines(totalYeonbong, -1, 17, color='gray', linestyle='solid', linewidth=3)
plt.text(17, (totalYeonbong - 190), '전국평균연봉', ha='right', va='center')

plt.grid(True, color = 'grey', linestyle=":")

plt.savefig("./data/시도별평균연봉.jpeg")

plt.show()

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
plt.figure(figsize=(10,4))

sidoCopy.sort_values('연봉대비거래금액', ascending=False, inplace=True)

plt.bar(sidoCopy['시도'], sidoCopy['연봉대비거래금액'], color='seagreen')

plt.xticks(rotation=70) 
plt.title("연봉 대비 거래금액")
plt.ylabel("연봉 대비 거래금액")

plt.grid(True, color = 'grey', linestyle=":")

plt.savefig("./data/연봉대비거래금액.jpeg")
plt.show()